# Starbucks Location Data

<p> We sourced worldwide Starbucks location <a href="https://www.kaggle.com/starbucks/store-locations"> data</a> from Kaggle.com. We then cleaned the data to only include locations in the United States and separated the postcode column into one with the 5 digit zipcode and another with the four extra digits. We kept only the 5 digit zipcode and loaded the information into a Postgres database.</p>

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import os

from config import password
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Read in Starbucks location data
url = "Resources/locations.csv"
locations_df = pd.read_csv(url)
locations_df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [3]:
# Keep only necessary columns in dataframe
locations_df = locations_df[["Store Number", "Store Name", "Street Address", 
                             "City", "State/Province", "Country", "Postcode", "Longitude", "Latitude"]]
locations_df.head()

,Store Number,Store Name,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude
0,47370-257954,"Meritxell, 96","Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,1.53,42.51
1,22331-212325,Ajman Drive Thru,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,55.47,25.42
2,47089-256771,Dana Mall,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,55.47,25.39
3,22126-218024,Twofour 54,Al Salam Street,Abu Dhabi,AZ,AE,NaN,54.38,24.48
4,17127-178586,Al Ain Tower,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,54.54,24.51


In [4]:
# Choose only locations in the United States
locations_df = locations_df[locations_df["Country"] == "US"]

In [5]:
# Cast postcode column as string 
locations_df["Postcode"] = locations_df["Postcode"].astype(str)
locations_df.dtypes

Store Number       object
Store Name         object
Street Address     object
City               object
State/Province     object
Country            object
Postcode           object
Longitude         float64
Latitude          float64
dtype: object

In [6]:
# Separate Postcode field to zipcode and zipcode+4 columns
locations_df["Zipcode"] = locations_df["Postcode"].map(lambda x: x[0:5])
locations_df["Zipcode + 4"] = locations_df["Postcode"].map(lambda x: x[5:9])
locations_df

,Store Number,Store Name,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude,Zipcode,Zipcode + 4
11964,3513-125945,Safeway-Anchorage #1809,5600 Debarr Rd Ste 9,Anchorage,AK,US,995042300,-149.78,61.21,99504,2300
11965,74352-84449,Safeway-Anchorage #2628,1725 Abbott Rd,Anchorage,AK,US,995073444,-149.84,61.14,99507,3444
11966,12449-152385,Safeway - Anchorage #1813,1501 Huffman Rd,Anchorage,AK,US,995153596,-149.85,61.11,99515,3596
11967,24936-233524,100th & C St - Anchorage,"320 W. 100th Ave, 100, Southgate Shopping Ctr ...",Anchorage,AK,US,99515,-149.89,61.13,99515,
11968,8973-85630,Old Seward & Diamond,1005 E Dimond Blvd,Anchorage,AK,US,995152050,-149.86,61.14,99515,2050
...,...,...,...,...,...,...,...,...,...,...,...
25567,74385-87621,Safeway-Laramie #2466,554 N 3rd St,Laramie,WY,US,820723012,-105.59,41.32,82072,3012
25568,73320-24375,Ridley's - Laramie #1131,3112 E. Grand,Laramie,WY,US,820705141,-105.56,41.31,82070,5141
25569,22425-219024,Laramie - Grand & 30th,3021 Grand Ave,Laramie,WY,US,82070,-105.56,41.31,82070,
25570,10849-103163,I-80 & Dewar Dr-Rock Springs,118 Westland Way,Rock Springs,WY,US,829015751,-109.25,41.58,82901,5751


In [7]:
# Drop postcode and last 4 of zipcode
locations_df = locations_df.drop(columns=["Postcode", "Zipcode + 4"])

In [8]:
# Rename columns to fit database structure and fillna's with empty string
clean_locs_df = locations_df[["Store Number", "Store Name", "Street Address", 
                             "City", "State/Province", "Country", "Zipcode", "Longitude", "Latitude"]]
clean_locs_df = clean_locs_df.rename(columns={"Store Number": "store_number", "Store Name": "store_name", 
                                              "Street Address":"street_address", "City":"city", "State/Province":"state", 
                                              "Country":"country", "Zipcode":"zipcode", 
                                              "Longitude":"longitude", "Latitude":"latitude"})
clean_locs_df["zipcode"] = clean_locs_df["zipcode"].astype(str) 
clean_locs_df["zipcode"] = clean_locs_df["zipcode"].fillna('')

In [9]:
# Export to csv 
clean_locs_df.to_csv("Clean_Data/clean_locs_df.csv")

## Connect to Local Database

In [10]:
rds_connection_string = ("postgres:{password}@localhost:5432/starbucks_etl".format(password=password))
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
engine.table_names()

['store_location', 'store_city']

In [12]:
#delete data from table before reload
engine.execute('''delete from store_location''')

In [13]:
clean_locs_df.to_sql(name="store_location", con=engine, if_exists="append", index=False)